# Creating sharded datasets with FAFB

### This section contains example of creating sharded skeletons, meshes with FAFB

### Import neccesary library modules now

In [1]:
import navis
import fafbseg
import pymaid

In [2]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

In [3]:
from pyroglancer.layers import create_nglayer
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.ngviewer import openviewer, closeviewer
from pyroglancer.ngspaces import create_ngspace
from pyroglancer.createconfig import createconfig

### Set configurations to fetch from data from CATMAID

In [4]:
publicurl = 'https://fafb.catmaid.virtualflybrain.org/'

In [5]:
working_rm = pymaid.CatmaidInstance(publicurl, api_token=None, project_id = 1)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
INFO - 2021-06-08 18:43:04,236 - client - Global CATMAID instance set. Caching is ON.


### Get sample skids and neuropil meshes from CATMAID

In [7]:
catmiad_neuronlist=pymaid.get_neuron('annotation:/\w+\sORN',
                                     remote_instance = working_rm) #get anything that matches ORN..

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO - 2021-06-08 18:43:19,096 - client - Cached data used. Use `pymaid.clear_cache()` to clear.


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO - 2021-06-08 18:43:19,167 - client - Cached data used. Use `pymaid.clear_cache()` to clear.


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO - 2021-06-08 18:43:19,201 - client - Cached data used. Use `pymaid.clear_cache()` to clear.
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO - 2021-06-08 18:43:19,289 - client - Cached data used. Use `pymaid.clear_cache()` to clear.


In [8]:
len(catmiad_neuronlist)

113

In [9]:
vols = pymaid.get_volume(['AL_L', 'AL_R'], color=(255, 0, 0, .2))

In [10]:
vols['AL_R'].id = 200
vols['AL_L'].id = 300
vols

{'AL_R': <navis.Volume(name=AL_R, id=200, color=(255, 0, 0, 0.2), vertices.shape=(622, 3), faces.shape=(1240, 3))>,
 'AL_L': <navis.Volume(name=AL_L, id=300, color=(255, 0, 0, 0.2), vertices.shape=(612, 3), faces.shape=(1228, 3))>}

### Start the dataserver to host precomputed data..

In [11]:
startdataserver()

Serving data from:  /var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh
Serving directory at http://127.0.0.1:8000


### Start a basic neuroglancer local session with all FAFB configurations

In [12]:
configdata = [dict(
             ngspace='FAFB',
             dimension=dict(x=1, y=1,z=1,units='um'),
             voxelsize=dict(x=4,y=4,z=40,units='nm'),
             layers=dict(
                    fafb_v14_clahe=dict(
                    type='image',
                    source='precomputed://gs://neuroglancer-fafb-data/fafb_v14/fafb_v14_clahe'),
                    fafb_surf=dict(
                    type='surfacemesh',
                    source='vtk://https://storage.googleapis.com/neuroglancer-fafb-data/elmr-data/FAFB.surf.vtk.gz'),
                    synapses_buhmann2019=dict(
                    type='synapsepred',
                    source='precomputed://gs://neuroglancer-20191211_fafbv14_buhmann2019_li20190805',
                    linkedseg='seg_20190805'),
                    seg_20190805=dict(
                    type='segmentation',
                    source='precomputed://gs://fafb-ffn1-20190805/segmentation',
                    ))
             )]

In [13]:
configfileloc = '/Users/sri/.pyroglancer/config_temp.yml'

In [14]:
createconfig(configdata, configfileloc, overwrite = True)

adding default config file..
setting default config file loc


In [15]:
layer_kws = {'ngspace': 'FAFB'}

In [16]:
create_ngspace(layer_kws)

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Neuroglancer viewer created at:  http://127.0.0.1:54637/v/91f086c683de8adaf3f3180a7021d8062ae199e7/
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  image
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Using layout : xy-3d
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  surfacemesh
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Using layout : xy-3d
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  synapsepred
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Using layout : xy-3d
config file 

### Add skids to neuroglancer layers..

In [17]:
tmpviewer = create_nglayer(layer_kws = {'type': 'skeletons',
                                        'source': catmiad_neuronlist,
                                        'name':'catmaid_skels',
                                        'sharding': True,
                                        'progress': True,
                                        'color': 'green',
                                        'alpha': 0.5})

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  skeletons


Synthesizing Shard Files: 100%|██████████| 77/77 [00:00<00:00, 22293.19it/s]

creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/catmaid_skels/skeletons/seg_props
Using layout : xy-3d



127.0.0.1 - - [08/Jun/2021 18:43:49] "GET /precomputed/catmaid_skels/skeletons/info HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 18:43:49] "GET /precomputed/catmaid_skels/skeletons/seg_props/info HTTP/1.1" 200 -


### Add synapses to neuroglancer layers..

In [18]:
tmpviewer = create_nglayer(layer_kws = {'type': 'synapses',
                                        'linked_layername': 'catmaid_skels',
                                        'source': catmiad_neuronlist})

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  synapses
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
flushing stuff..
presynapse stuff at: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh /precomputed/catmaid_skels/presynapses
postsynapse stuff at: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh /precomputed/catmaid_skels/postsynapses
synapses info path: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/catmaid_skels
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/catmaid_skels/presynapses
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/catmaid_skels/postsynapses
Adding neuron:  12013573
making: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/prec

127.0.0.1 - - [08/Jun/2021 18:43:52] "GET /precomputed/catmaid_skels/presynapses/info HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 18:43:52] "GET /precomputed/catmaid_skels/postsynapses/info HTTP/1.1" 200 -


### Add neuropil meshes to neuroglancer layers..

In [19]:
tmpviewer = create_nglayer(layer_kws = {'type': 'volumes','source': [vols['AL_R'],vols['AL_L']],
                                        'name': 'neuropils','color': ['magenta', 'blue'], 'alpha': 0.3,
                                        'multires': True, 
                                        'sharding': True,
                                        'progress': True})

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  volumes
mesh/200
Seg id is: 200
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/neuropils/mesh/200
Vol id is: 200
mesh/300
Seg id is: 300
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/neuropils/mesh/300
Vol id is: 300


Synthesizing Shard Files: 100%|██████████| 1/1 [00:00<00:00, 5714.31it/s]

creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/neuropils/mesh/segment_properties
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/neuropils/mesh/segment_names
Using layout : xy-3d



127.0.0.1 - - [08/Jun/2021 18:44:01] "GET /precomputed/neuropils/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 18:44:01] "GET /precomputed/neuropils/mesh/segment_properties/info HTTP/1.1" 200 -


### Add annotations meshes to neuroglancer layers..

In [20]:
temp_pts = pd.DataFrame([[123072, 47001, 3375]],columns=['x','y','z'])
temp_pts = pd.DataFrame([[123072, 47001, 3375], [120000, 17001, 3000]], columns=['x', 'y', 'z'])
temp_pts['description'] = ['center_pt','above_pt']

In [21]:
#plot landmarks..
tmpviewer = create_nglayer(layer_kws = {'type': 'points','name': 'landmarks',
                                        "annotationstatetype": 'precomputed',
                                        'source': temp_pts,'color': 'orange'})

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  points
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
using  voxel space with scale:  [4, 4, 40]
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/landmarks
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/landmarks/spatial0/0_0_0
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/landmarks/by_id/0
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh/precomputed/landmarks/by_id/1
Using layout : xy-3d


127.0.0.1 - - [08/Jun/2021 18:44:06] "GET /precomputed/landmarks/info HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 18:44:09] code 404, message File not found
127.0.0.1 - - [08/Jun/2021 18:44:09] "GET /precomputed/catmaid_skels/presynapses/spatial0/0_0_0 HTTP/1.1" 404 -
127.0.0.1 - - [08/Jun/2021 18:44:09] code 404, message File not found
127.0.0.1 - - [08/Jun/2021 18:44:09] "GET /precomputed/catmaid_skels/postsynapses/spatial0/0_0_0 HTTP/1.1" 404 -
127.0.0.1 - - [08/Jun/2021 18:44:09] "GET /precomputed/landmarks/spatial0/0_0_0 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 18:44:20] "OPTIONS /precomputed/neuropils/mesh/22b9.shard HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 18:44:20] "OPTIONS /precomputed/neuropils/mesh/22b9.shard HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 18:44:20] "GET /precomputed/neuropils/mesh/22b9.shard HTTP/1.1" 206 -
127.0.0.1 - - [08/Jun/2021 18:44:20] "OPTIONS /precomputed/neuropils/mesh/22b9.shard HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 18:44:20] "OPTIONS /precom

### Screenshot of the neuroglancer instance should like below..

#### The grey mesh is the v14 neuropil, magenta is the Antenna lobe (Right), blue is the Antenna lobe (left), you can see the skeletons from CATMAID (green) plotted in the skeleton layer, along with the corresponding synapses (red-presynapses, blue-postsynapses) and some landmark points plotted in yellow

![alt text](pics/shardeddataset_fafb.png "Title")

## Step 10: Close the viewer and dataserver..

In [22]:
closeviewer()

closing already existing ng viewer


In [23]:
closedataserver()

Closing server at http://127.0.0.1:8000
Cleaning directory at /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpyeyfi0qh
